In [ ]:
!pip install -q transformers torch scikit-learn pandas

import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer
from transformers import TrainingArguments
print("TrainingArguments class is from:", TrainingArguments.__module__)
from transformers import pipeline

import transformers
print(transformers.__version__)

!pip install -q --upgrade transformers
import torch
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

from google.colab import drive
drive.mount('/content/drive')

# CSV dosyasını oku
csv_file_path = '/content/drive/My Drive/clinical_notes.csv'
df = pd.read_csv(csv_file_path)

# Tek örnekli sınıfları çıkar (Akut Miyokart Enfarktüsü gibi)
label_counts = df['etiket'].value_counts()
df = df[df['etiket'].isin(label_counts[label_counts >= 2].index)]

# Etiketleri sayısal değerlere dönüştür
unique_labels = df['etiket'].unique()
label_to_id = {label: i for i, label in enumerate(unique_labels)}
id_to_label = {i: label for i, label in enumerate(unique_labels)}

df['etiket_id'] = df['etiket'].map(label_to_id)

print(df['etiket'].value_counts())

# Model için kullanılacak metin ve etiket listelerini hazırla
texts = df['not'].tolist()
labels = df['etiket_id'].tolist()

# Eğitim ve test setlerine ayırma
train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=42, stratify=labels
)

print(f"Train sayısı: {len(train_texts)}")
print(f"Validation verisi sayısı: {len(val_texts)}")
print(f"Toplam etiket sayısı: {len(unique_labels)}")
print(f"Etiket eşleşmeleri: {label_to_id}")

# --- 2. Model ve Tokenizer Yükleme ---
print("\n--- 2. Model ve Tokenizer Yükleniyor ---")

model_name = "dbmdz/bert-base-turkish-cased" # BERTurk model adı

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(unique_labels))

# GPU varsa modeli GPU'ya tasima
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print(f"Kullanılan cihaz: {device}")

# eri Seti Sınıfı Oluşturma
print("\n~ Veri Seti Sınıfı Oluşturuluyor ~")

class ClinicalNotesDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Metinleri token'lara ayırma
train_encodings = tokenizer(train_texts, truncation=True, padding='max_length', max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding='max_length', max_length=128)

train_dataset = ClinicalNotesDataset(train_encodings, train_labels)
val_dataset = ClinicalNotesDataset(val_encodings, val_labels)

# --- 4. Metrik Fonksiyonu Tanımlama ---
print("\n--- 4. Metrik Fonksiyonu Tanımlanıyor ---")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)

    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted', zero_division=0)
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

# --- 5. Fine-tuning için Eğitim Argümanları ve Trainer ---
print("\n--- 5. Fine-tuning Başlatılıyor ---")

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=50,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# Modeli Eğitme
trainer.train()

# Eğitim tamamlandıktan sonra en iyi modeli kaydetme
model_save_path = "./fine_tuned_clinical_berturk"
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)

print(f"Fine-tuning tamamlandı ve model '{model_save_path}' klasörüne kaydedildi.")

metrics = trainer.evaluate()
print("Final validation metrics:", metrics)

# --- 6. Eğitilmiş Modelin Kullanımı ---
print("\n~ Eğitilmiş Model Test Ediliyor ~")

# Eğitilmiş modeli ve tokenizer'ı yükleyelim
loaded_tokenizer = AutoTokenizer.from_pretrained(model_save_path)
loaded_model = AutoModelForSequenceClassification.from_pretrained(model_save_path)

# Tahmin için pipeline oluşturalım
classifier = pipeline(
    "text-classification",
    model=loaded_model,
    tokenizer=loaded_tokenizer,
    device=0 if torch.cuda.is_available() else -1
)

print("\n~ Test Seti ile Değerlendirme ~")

# Test CSV'yi oku
test_df = pd.read_csv("/content/drive/MyDrive/clinical_notes_test.csv")
test_df = test_df.dropna()

# Etiketleri encode et (aynı label_to_id kullanılarak)
test_df = test_df[test_df['etiket'].isin(label_to_id.keys())]
test_df['etiket_id'] = test_df['etiket'].map(label_to_id)

# Tokenization
test_encodings = tokenizer(
    test_df['not'].tolist(),
    truncation=True,
    padding='max_length',
    max_length=128
)

# Dataset objesi
test_dataset = ClinicalNotesDataset(test_encodings, test_df['etiket_id'].tolist())

# Tahminleri al
predictions = trainer.predict(test_dataset)
pred_labels = np.argmax(predictions.predictions, axis=1)

# Gerçek etiketler
true_labels = test_df['etiket_id'].tolist()

# Skorları hesapla
acc = accuracy_score(true_labels, pred_labels)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, pred_labels, average='weighted', zero_division=0)

print("~ Test Seti Sonuçları:")
print(f"Accuracy:  {acc:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-score:  {f1:.4f}")


TrainingArguments class is from: transformers.training_args
4.53.3
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
etiket
Diyabet                         69
Hipertansiyon                   55
Depresyon                       48
KOAH                            47
Üst Solunum Yolu Enfeksiyonu    47
Menenjit                        46
Kronik Böbrek Yetmezliği        46
Astım                           44
Karaciğer Yetmezliği            44
Sağlıklı                        37
Anemi                           16
Name: count, dtype: int64
Eğitim verisi sayısı: 399
Doğrulama verisi sayısı: 100
Toplam etiket sayısı: 11
Etiket eşleşmeleri: {'Diyabet': 0, 'Hipertansiyon': 1, 'KOAH': 2, 'Üst Solunum Yolu Enfeksiyonu': 3, 'Karaciğer Yetmezliği': 4, 'Kronik Böbrek Yetmezliği': 5, 'Astım': 6, 'Anemi': 7, 'Menenjit': 8, 'Depresyon': 9, 'Sağlıklı': 10}

--- 2. Model ve Tokenizer Yükleniyor ---


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Kullanılan cihaz: cpu

~ Veri Seti Sınıfı Oluşturuluyor ~

--- 4. Metrik Fonksiyonu Tanımlanıyor ---

--- 5. Fine-tuning Başlatılıyor ---


Step,Training Loss
100,1.230300
200,0.014400
300,0.003900
400,0.002800
